In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..

/home/tejomay/cgpos


In [3]:
import random
import sys

import torch

import src.config as cfg
from src.util import read_pkl

def display_bar(data, line_len=50):
    out = ""
    for i, v in enumerate(data):
        if i % 50 == 0:
            out += "\n"
        match v:
            case 1:
                out += "\033[42m \033[0m"
            case 0:
                out += "\033[41m \033[0m"
    sys.stdout.write(out)            
    sys.stdout.flush()

In [4]:
random.seed(20)
n_chunks = 5000
train_size = 0.98

data = read_pkl(cfg.ft_syl)
vocab = sorted(set(data))
vocab_size = len(vocab)

tok2int = {ch: i for i, ch in enumerate(vocab)}
int2tok = {i: ch for ch, i in tok2int.items()}
encode = lambda text: [tok2int[c] for c in text]
decode = lambda tokens: "".join([int2tok[i] for i in tokens])
tokens = torch.tensor(encode(data), dtype=torch.long)

chunks = torch.split(tokens, len(data) // (n_chunks - 1))
l = [1] * int(n_chunks * train_size) + [0] * int(n_chunks * (1 - train_size))
random.shuffle(l)
train_data = torch.cat([chunks[i] for i, v in enumerate(l) if v])
val_data = torch.cat([chunks[i] for i, v in enumerate(l) if not v])

display_bar(l)